In [42]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [43]:
iris=load_iris()
features=np.array(iris.data)
target=np.array(iris.target)

In [44]:
indices=np.random.permutation(len(features))
features=features[indices]
target=target[indices]
print(f"targets are \n\n{target}  \n\nfeatures are \n\n{features}")
train_size=int(0.8*len(features))
X_train=features[:train_size]
y_train=target[:train_size]
X_test=features[train_size:]
y_test=target[train_size:]

targets are 

[2 2 1 2 1 1 2 1 0 0 2 2 1 0 2 1 2 0 0 1 1 1 1 1 1 1 0 1 2 1 2 2 0 2 1 1 2
 2 1 2 0 1 1 1 0 0 1 2 0 2 1 2 1 1 0 0 2 1 2 0 1 2 0 2 0 2 0 1 2 0 2 2 1 0
 0 2 2 0 0 0 0 1 2 2 2 1 2 0 1 2 0 0 2 1 0 0 2 0 1 0 1 0 1 0 0 1 2 2 0 2 0
 2 0 0 2 1 2 1 1 1 2 0 0 0 2 1 2 0 2 0 2 1 1 2 1 0 2 1 2 0 2 1 0 0 0 1 1 1
 0 0]  

features are 

[[5.7 2.5 5.  2. ]
 [6.4 2.7 5.3 1.9]
 [6.7 3.  5.  1.7]
 [5.8 2.7 5.1 1.9]
 [6.2 2.2 4.5 1.5]
 [6.1 2.9 4.7 1.4]
 [7.7 3.8 6.7 2.2]
 [6.1 3.  4.6 1.4]
 [5.1 3.8 1.5 0.3]
 [4.8 3.4 1.6 0.2]
 [6.3 2.9 5.6 1.8]
 [6.5 3.2 5.1 2. ]
 [5.7 2.8 4.5 1.3]
 [4.8 3.  1.4 0.1]
 [6.7 3.3 5.7 2.1]
 [5.5 2.3 4.  1.3]
 [6.  2.2 5.  1.5]
 [5.  3.5 1.3 0.3]
 [4.3 3.  1.1 0.1]
 [5.  2.3 3.3 1. ]
 [5.6 2.9 3.6 1.3]
 [6.7 3.1 4.4 1.4]
 [5.1 2.5 3.  1.1]
 [4.9 2.4 3.3 1. ]
 [5.6 3.  4.1 1.3]
 [5.7 2.6 3.5 1. ]
 [4.7 3.2 1.6 0.2]
 [5.5 2.6 4.4 1.2]
 [6.7 3.  5.2 2.3]
 [5.2 2.7 3.9 1.4]
 [6.5 3.  5.5 1.8]
 [5.6 2.8 4.9 2. ]
 [5.7 4.4 1.5 0.4]
 [7.2 3.  5.8 1.6]
 [5.4 3.  4.5 1.

In [52]:
class Node:
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,info_gain=None,value=None):
        self.feature_index=feature_index
        self.threshold=threshold
        self.left=left
        self.right=right
        self.info_gain=info_gain
        self.value=value

In [53]:
class DecisionTreeClassifier:
    def __init__(self,min_samples_split=2,max_depth=2):
        self.root=None
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
    def build_tree(self, X, y, curr_depth=0):
        num_samples, num_features = X.shape
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(X, y, num_samples, num_features)
            if best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"][:, :-1], best_split["dataset_left"][:, -1], curr_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"][:, :-1], best_split["dataset_right"][:, -1], curr_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(y)
        return Node(value=leaf_value)
    def get_best_split(self, X, y, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = X[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(X, y, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    info_gain = self.information_gain(y, dataset_left[:, -1], dataset_right[:, -1])
                    if info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = info_gain
                        max_info_gain = info_gain
        return best_split
    def split(self, X, y, feature_index, threshold):
        left_idxs = X[:, feature_index] <= threshold
        right_idxs = X[:, feature_index] > threshold
        dataset_left = np.concatenate([X[left_idxs], y[left_idxs].reshape(-1, 1)], axis=1)
        dataset_right = np.concatenate([X[right_idxs], y[right_idxs].reshape(-1, 1)], axis=1)
        return dataset_left, dataset_right
    def information_gain(self,parent,l_child,r_child,mode="entropy"):
        weight_l=len(l_child)/len(parent)
        weight_r=len(r_child)/len(parent)
        if mode=="gini":
            gain=self.gini_index(parent)-(weight_l*self.gini_index(l_child)+weight_r*self.gini_index(r_child))
        else:
            gain=self.entropy(parent)-(weight_l*self.entropy(l_child)+weight_r*self.entropy(r_child))
        return gain
    def entropy(self,y):
        class_labels=np.unique(y)
        entropy=0
        for cls in class_labels:
            p_cls=len(y[y==cls])/len(y)
            entropy+=-p_cls*np.log2(p_cls)
        return entropy
    def gini_index(self,y):
        class_labels=np.unique(y)
        entropy=0
        for cls in class_labels:
            p_cls=len(y[y==cls])/len(y)
            gini+=p_cls**2
        return gini
    def calculate_leaf_value(self, y):
        values, counts = np.unique(y, return_counts=True)
        # Return the most common class value
        return values[np.argmax(counts)]

    def print_tree(self, node=None, depth=0, feature_names=None):
        if node is None:
            node = self.root
    
        if node.value is not None:  # If it's a leaf node
            print("\t" * depth + "Predict", node.value)
        else:
            if feature_names:
                feature_name = feature_names[node.feature_index]
            else:
                feature_name = f"Feature[{node.feature_index}]"
            print("\t" * depth + f"{feature_name} <= {node.threshold}? (Info gain: {node.info_gain:.3f})")
            print("\t" * depth + "-> True:")
            self.print_tree(node.left, depth + 1, feature_names)
            print("\t" * depth + "-> False:")
            self.print_tree(node.right, depth + 1, feature_names)

    def fit(self,X,y):
        self.root=self.build_tree(X,y)
    def predict(self,X):
        predictions=[self.make_prediction(x,self.root) for x in X]
        return predictions
    def make_prediction(self,x,tree):
        if tree.value!=None:return tree.value
        feature_val=x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x,tree.left)
        else:
            return self.make_prediction(x,tree.right)
    def calculate_accuracy(self, y_true, X):
        y_pred = self.predict(X)
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

In [54]:
classifier=DecisionTreeClassifier(min_samples_split=2,max_depth=10)
classifier.fit(X_train,y_train)
classifier.print_tree()
print(f"Accuracy of decision tree is { classifier.calculate_accuracy(y_test,X_test)}")

Feature[2] <= 1.9? (Info gain: 0.901)
-> True:
	Predict 0.0
-> False:
	Feature[2] <= 4.7? (Info gain: 0.727)
	-> True:
		Feature[3] <= 1.6? (Info gain: 0.172)
		-> True:
			Predict 1.0
		-> False:
			Predict 2.0
	-> False:
		Feature[3] <= 1.7? (Info gain: 0.205)
		-> True:
			Feature[1] <= 2.6? (Info gain: 0.292)
			-> True:
				Predict 2.0
			-> False:
				Feature[2] <= 5.0? (Info gain: 0.420)
				-> True:
					Predict 1.0
				-> False:
					Feature[0] <= 6.0? (Info gain: 0.918)
					-> True:
						Predict 1.0
					-> False:
						Predict 2.0
		-> False:
			Predict 2.0
Accuracy of decision tree is 0.9333333333333333


In [55]:
from graphviz import Digraph

def visualize_tree(node, dot=None):
    if dot is None:
        dot = Digraph()
    
    if node.value is None:
        dot.node(name=str(id(node)), label=f"Feature {node.feature_index} > {node.threshold}\\nInfo Gain: {node.info_gain}")
    else:
        dot.node(name=str(id(node)), label=f"Leaf: {node.value}")

    if node.left is not None:
        visualize_tree(node.left, dot)
        dot.edge(str(id(node)), str(id(node.left)))

    if node.right is not None:
        visualize_tree(node.right, dot)
        dot.edge(str(id(node)), str(id(node.right)))

    return dot

def plot_tree(tree):
    dot = visualize_tree(tree.root)
    dot.view()

plot_tree(classifier)